# Chapter 8 Exercise Answer*

#### \*) all answers are not checked, please make a pull request if the answer is incorrect.

In [6]:
TMP_FILE=('tmp_cp8no1.sh' 'tmp_cp8no2.sh' 'tmp_cp8no3.sh' 'tmp_cp8no4.sh', 'tmp_cp8no5.sh')

## No. 1

In [14]:
tee ${TMP_FILE[0]} << EOF
#!/usr/bin/env bash
# Because jupyter notebook can't receive prompt, I will bypass it using $1
# So, you can test it using ./${TMP_FILE[0]} [age]

# Minimum age for consuming alcohol
MIN_AGE=16

if [[ -z \$1 ]]; then 
    # Prompt user's age
    read -p "Input your age: " USER_AGE
else
    USER_AGE=\$1
fi

if [[ \${USER_AGE} -lt \${MIN_AGE} ]]; then
    echo "You need to wait \$(( \${MIN_AGE} - \${USER_AGE} )) year(s) to consume this hell water."
else
    echo "You are alowed to drink this hell water."
fi

# Statistic about 18+ alcoholic user
# TODO Here
EOF

# Give +x permission to script
chmod +x ${TMP_FILE[0]}

#!/usr/bin/env bash
# Because jupyter notebook can't receive prompt, I will bypass it using 
# So, you can test it using ./tmp_cp8no1.sh [age]

# Minimum age for consuming alcohol
MIN_AGE=16

if [[ -z $1 ]]; then 
    # Prompt user's age
    read -p "Input your age: " USER_AGE
else
    USER_AGE=$1
fi

if [[ ${USER_AGE} -lt ${MIN_AGE} ]]; then
    echo "You need to wait $(( ${MIN_AGE} - ${USER_AGE} )) year(s) to consume this hell water."
else
    echo "You are alowed to drink this hell water."
fi

# Statistic about 18+ alcoholic user
# TODO Here


In [13]:
# Test it
./${TMP_FILE[0]} 10

You need to wait 6 year(s) to consume this hell water.


## No. 2

In [128]:
# Create script
tee ${TMP_FILE[1]} << EOF
#!/usr/bin/env bash

AVAILABLE_COMMAND=(gzip bzip2 compress zip)

# Ensure $1 is present
if [[ -z \$1 ]]; then
    echo "Usage: ./${TMP_FILE[1]} [file1] [file2] [file-n] [compression-command]"
    echo "Available command: gzip, bzip2, compress, zip."
    exit 1
fi

function is_contain_command() {
    for i in \${AVAILABLE_COMMAND[@]}; do
        if [[ \${i} == \$1 ]]; then
            return 0
        fi
    done
    return 1
}

# If user already specify compression-command from cli, then just set it.
if [[ -z \${@:\$#} ]]; then
    echo "Which one do you want to use?"
    # Use here document with <<-, to allow identation, but
    # jupyter use space to indent, so I will place the DELIM to the first of line.
    cat <<-DELIM
        gzip
        bzip2
        compress
        zip
DELIM
    read -p "Your choice: " USER_CHOICE
else
    # Get last parameter, it also can use \${@: -1}
    USER_CHOICE="\${@:\$#}"
    if ! is_contain_command "\${USER_CHOICE}" ; then
        echo "Unknown compression command."
    else
        echo "\${USER_CHOICE} selected."
    fi
fi

# exec user choice
\${USER_CHOICE} \${@:1:\$#-1}
EOF

# Grant +x on the script
chmod +x ${TMP_FILE[1]}

#!/usr/bin/env bash

AVAILABLE_COMMAND=(gzip bzip2 compress zip)

# Ensure  is present
if [[ -z $1 ]]; then
    echo "Usage: ./tmp_cp8no2.sh [file1] [file2] [file-n] [compression-command]"
    echo "Available command: gzip, bzip2, compress, zip."
    exit 1
fi

function is_contain_command() {
    for i in ${AVAILABLE_COMMAND[@]}; do
        if [[ ${i} == $1 ]]; then
            return 0
        fi
    done
    return 1
}

# If user already specify compression-command from cli, then just set it.
if [[ -z ${@:$#} ]]; then
    echo "Which one do you want to use?"
    # Use here document with <<-, to allow identation, but
    # jupyter use space to indent, so I will place the DELIM to the first of line.
    cat <<-DELIM
        gzip
        bzip2
        compress
        zip
DELIM
    read -p "Your choice: " USER_CHOICE
else
    # Get last parameter, it also can use ${@: -1}
    USER_CHOICE="${@:$#}"
    if ! is_contain_command "${USER_CHOICE}" ; then
        echo "Unknown compression comm

In [127]:
# Test it
./${TMP_FILE[1]} test test2 test3 gzip

gzip selected.
gzip: data2018-05-25: No such file or directory
gzip: test: No such file or directory
gzip: test2: No such file or directory
gzip: test3: No such file or directory


: 1

## No. 3

In [7]:
# Create script
tee ${TMP_FILE[2]} << EOF
#!/usr/bin/env bash
BACKUP_FOLDER='/var/backups'
BACKUP_PREFIX=('full-bak' 'partial-bak')

# Make sure this script doesn't expect any argument.
if [[ ! -z \$1 ]]; then
    echo "Usage: ${TMP_FILE[2]}"
fi

# Make backup folder if not exists
if [[ ! -d \${BACKUP_FOLDER} ]]; then sudo mkdir -p \${BACKUP_FOLDER}; fi

# Check current home directory size is lower than the available size.
# If higher than the available size on backup directory, so the script will abort the action.
function check_home_size() {
    # Use sudo if there is root folder inside home directory. Remove it if not urgently needed.
    HOME_DIR_SIZE=\$(du -cs ~ | grep total | awk '{print $1}')
    AVAILABLE_BACKUP_SIZE=\$(df \${BACKUP_FOLDER} | tail -n 1 | awk '{print $4}')
    
    if (( \${HOME_DIR_SIZE} < \${AVAILABLE_BACKUP_SIZE} )); then
        return 0
    fi
    
    return 1
}

# Ask user want to use incremental backup or full backup
if ! ls \${BACKUP_FOLDER} | grep \${BACKUP_PREFIX}; then
    do_backup full
else
    read -p "Do you want a full backup or incremental backup (if full backup is no longer than one week)? (full/incremental)[default: full]" BACKUP_TYPE
    # Todo Here: Research about incremental backup.
    case "\${BACKUP_TYPE}" in
        full)
            do_backup full
            ;;
        
        incremental)
            do_backup incremental
            ;;
            
        *)
            do_backup full
            ;;
    esac
fi

function do_backup() {
    BACKUP_TYPE="\$1"
    BACKUP_TIMESTAMP=\`date "+%s"\`
    TAR=\`which tar\`
    echo "Please wait... This process takes much time as much your directory space."
    if [[ \${BACKUP_TYPE} == "full" ]]; then
        CURRENT_FILE="\${BACKUP_PREFIX[0]}-\${BACKUP_TIMESTAMP}.tar.gz"
        \${TAR} --listed-incremental "\${BACKUP_FOLDER}/backup.snar" -czpf "\${CURRENT_FILE}" "/home/\$USER" 
        cp -f "\${BACKUP_FOLDER}/backup.snar" "\${BACKUP_FOLDER}/backup-1.snar" 
    elif [[ ${BACKUP_TYPE} == "incremental" ]]; then
        # Ignore this action if full backup is aged for more than a week.
        FULL_BACKUP_LIST=\`ls --time-style=+%s -lat \${BACKUP_FOLDER} | grep \${BACKUP_PREFIX[0]} | awk -v LAST_WEEK=\`date --date="1 week ago" +%s\` '\$6 > LAST_WEEK{print \$7}'\`
        if [[ ! -z \${FULL_BACKUP_LIST} ]]; then
            CURRENT_FILE="\${BACKUP_PREFIX[1]}-\${BACKUP_TIMESTAMP}.tar.gz"
            \${TAR} --listed-incremental "\${BACKUP_FOLDER}/backup-1.snar" -czpf "\${CURRENT_FILE}" "/home/\$USER" 
        else
            do_backup full
        fi
    fi
    
    echo "Compressed size: \$(wc -c < \${CURRENT_FILE})"
}
EOF

# Give permission
chmod +x ${TMP_FILE[2]}

#!/usr/bin/env bash
BACKUP_FOLDER='/var/backups'
BACKUP_PREFIX=('full-bak' 'partial-bak')

# Make sure this script doesn't expect any argument.
if [[ ! -z $1 ]]; then
    echo "Usage: tmp_cp8no3.sh"
fi

# Make backup folder if not exists
if [[ ! -d ${BACKUP_FOLDER} ]]; then sudo mkdir -p ${BACKUP_FOLDER}; fi

# Check current home directory size is lower than the available size.
# If higher than the available size on backup directory, so the script will abort the action.
function check_home_size() {
    # Use sudo if there is root folder inside home directory. Remove it if not urgently needed.
    HOME_DIR_SIZE=$(du -cs ~ | grep total | awk '{print }')
    AVAILABLE_BACKUP_SIZE=$(df ${BACKUP_FOLDER} | tail -n 1 | awk '{print }')
    
    if (( ${HOME_DIR_SIZE} < ${AVAILABLE_BACKUP_SIZE} )); then
        return 0
    fi
    
    return 1
}

# Ask user want to use incremental backup or full backup
if ! ls ${BACKUP_FOLDER} | grep ${BACKUP_PREFIX}; then
    do_backup full
else
    read -p "

## No. 4

In [1]:
# Create script
tee ${TMP_FILE[3]} << EOF
#!/usr/bin/env bash
# Only one argument is accepted
if (( \$\# > 1 )); then
    echo "Usage: ${TMP_FILE[3]} [username]"
    exit 1
fi

# Make sure \$1 is not empty
if [[ -z \$1 ]]; then
    echo "Username can't be empty"
fi
USERNAME="\$1"
PASSWORD="1234"
useradd -m \$USERNAME

if [[ $? == 0 ]]; then
    # Set default password
    echo \$USERNAME:\$PASSWORD | chpasswd
fi

echo "Your default password: \$PASSWORD"
echo "Your UID: \$(id \$USERNAME -u) and GID: \$(id \$USERNAME -g)"
echo "Your home directory is at: /home/\$USERNAME"
exit 0
EOF

# Grant +x permission
chmod +x ${TMP_FILE[3]}

#!/usr/bin/env bash
# Only one argument is accepted
if (( $\# > 1 )); then
    echo "Usage:  [username]"
    exit 1
fi

# Make sure $1 is not empty
if [[ -z $1 ]]; then
    echo "Username can't be empty"
fi
USERNAME="$1"
PASSWORD="1234"
useradd -m $USERNAME

if [[ 1 == 0 ]]; then
    # Set default password
    echo $USERNAME:$PASSWORD | chpasswd
fi

echo "Your default password: $PASSWORD"
echo "Your UID: $(id $USERNAME -u) and GID: $(id $USERNAME -g)"
echo "Your home directory is at: /home/$USERNAME"
exit 0
chmod: missing operand after ‘+x’
Try 'chmod --help' for more information.


: 1

## No. 5

In [5]:
# Add script
tee ${TMP_FILE[4]} << EOF

#!/bin/bash

# This script gives information about a file.

read -p "Enter a filename (ex. /home/user1/.bashrc): " FILENAME

echo "Properties for \$FILENAME:"

if [ -f \$FILENAME ]; then
  echo "Size is \$(ls -lh \$FILENAME | awk '{ print $5 }')"
  echo "Type is \$(file \$FILENAME | cut -d":" -f2 -)"
  echo "Inode number is \$(ls -i \$FILENAME | cut -d" " -f1 -)"
  echo "\$(df -h \$FILENAME | grep -v Mounted | awk '{ print "On",\$1", \
which is mounted as the",\$6,"partition."}')"
else
  echo "File does not exist."
fi
EOF

# Grant +x permission
chmod +x ${TMP_FILE[4]}


#!/bin/bash

# This script gives information about a file.

read -p "Enter a filename (ex. /home/user1/.bashrc): " FILENAME

echo "Properties for $FILENAME:"

if [ -f $FILENAME ]; then
  echo "Size is $(ls -lh $FILENAME | awk '{ print  }')"
  echo "Type is $(file $FILENAME | cut -d":" -f2 -)"
  echo "Inode number is $(ls -i $FILENAME | cut -d" " -f1 -)"
  echo "$(df -h $FILENAME | grep -v Mounted | awk '{ print "On",$1", which is mounted as the",$6,"partition."}')"
else
  echo "File does not exist."
fi


In [6]:
# Test it
./${TMP_FILE[4]}

Enter a filename (ex. /home/user1/.bashrc): 

### Removal Script

In [ ]:
for i in ${TMP_FILE[@]}; do
    rm ${i} && echo "File ${i} has been successfully deleted."
done